In [ ]:
import ISLP
from sklearn.model_selection import train_test_split
#Load and split dataset
Carseats = ISLP.load_data('Carseats')
train_data, test_data = train_test_split(Carseats, test_size=0.2, random_state=0)
train_data.columns

In [ ]:
#Remove data we are not using for the multiregression
CS = train_data.columns.drop(['Sales', 'CompPrice', 'Income', 'Advertising', 'Population',
       'ShelveLoc', 'Age', 'Education'])
train_data[CS]

In [61]:
from ISLP.models import ModelSpec as MS
from ISLP.models import summarize
import statsmodels.api as sm

#perform regression
y_train = train_data['Sales']
X_train = MS(CS).fit_transform(train_data)
model = sm.OLS(y_train, X_train)
result = model.fit()
summarize(result)

,coef,std err,t,P>|t|
intercept,13.2766,0.737,18.008,0.000
Price,-0.0566,0.006,-9.489,0.000
Urban[Yes],-0.0700,0.304,-0.230,0.818
US[Yes],1.1762,0.290,4.059,0.000


In [65]:
import numpy as np

#validate on test data using mean squared error
X_test = MS(CS).fit_transform(test_data)
pred = result.get_prediction(X_test)
np.mean((pred.predicted-test_data['Sales'])**2)

5.696443390432015

In [63]:
#improve regression by removing the Urban predictor
CS_better = [CS[0], CS[2]]
X_better = MS(CS_better).fit_transform(train_data)
model_better = sm.OLS(y_train, X_better)
result_better = model_better.fit()
summarize(result_better)

,coef,std err,t,P>|t|
intercept,13.2411,0.720,18.392,0.0
Price,-0.0567,0.006,-9.541,0.0
US[Yes],1.1727,0.289,4.059,0.0


In [67]:
#validate on test data using mean squared error
X_test = MS(CS_better).fit_transform(test_data)
pred_better = result_better.get_prediction(X_test)
np.mean((pred_better.predicted-test_data['Sales'])**2)

5.690393290449474

In [68]:
result_better.conf_int(alpha = 0.05)

,0,1
intercept,11.824687,14.657605
Price,-0.068405,-0.045017
US[Yes],0.604248,1.741125
